In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_csv('train_mod.csv')

In [17]:
y = df.iloc[:,1].values
X = df[[u'Pclass',u'Sex', u'Age',u'SibSp', u'Parch',u'Fare']].values

In [19]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe_lr = Pipeline([('std',StandardScaler()),('lr',LogisticRegression(random_state=0))])
pipe_lr.fit(X_train,y_train)
print(pipe_lr.score(X_test,y_test))

0.798507462687


In [31]:
from sklearn.cross_validation import StratifiedKFold

kfold = StratifiedKFold(y=y_train,n_folds=10,random_state=1)
scores = []
for k, (train,test) in enumerate(kfold):
    pipe_lr.fit(X_train[train],y_train[train])
    score = pipe_lr.score(X_train[test],y_train[test])
    scores.append(score)
    print('Fold %s,Class dist: %s, Acc:%.3f') %(k+1,np.bincount(y_train[train]),score)

Fold 1,Class dist: [342 217], Acc:0.766
Fold 2,Class dist: [343 217], Acc:0.810
Fold 3,Class dist: [343 218], Acc:0.758
Fold 4,Class dist: [343 218], Acc:0.758
Fold 5,Class dist: [343 218], Acc:0.758
Fold 6,Class dist: [343 218], Acc:0.758
Fold 7,Class dist: [343 218], Acc:0.806
Fold 8,Class dist: [343 218], Acc:0.823
Fold 9,Class dist: [343 218], Acc:0.855
Fold 10,Class dist: [343 218], Acc:0.839


In [33]:
print('CV accyracy: %.3f +/- %.3f') %(np.mean(scores),np.std(scores))

CV accyracy: 0.793 +/- 0.036


In [41]:
from sklearn.cross_validation import cross_val_score

scores = cross_val_score(estimator=pipe_lr,X=X_train,y=y_train,cv=10,n_jobs=-1)
print('CV sccuracy scores:%s') % scores
print('CV accuracy: %.3f +/- %.3f') %(np.mean(scores),np.std(scores))

CV sccuracy scores:[ 0.765625    0.80952381  0.75806452  0.75806452  0.75806452  0.75806452
  0.80645161  0.82258065  0.85483871  0.83870968]
CV accuracy: 0.793 +/- 0.036


In [46]:
from sklearn.grid_search import GridSearchCV
params_range = [10**(i) for i in np.arange(-4,5,1)]
param_grid =[{'lr__C':params_range,'lr__penalty':['l1']},
            {'lr__C':params_range,'lr__penalty':['l2']}]

gs = GridSearchCV(estimator=pipe_lr,
                 param_grid=param_grid,
                 scoring='accuracy',
                 cv=10,n_jobs=-1)

gs = gs.fit(X_train,y_train)
print(gs.best_score_)
print(gs.best_params_)

0.792937399679
{'lr__penalty': 'l1', 'lr__C': 1}


In [48]:
from sklearn.svm import SVC
pipe_svc = Pipeline([('scl',StandardScaler()),('clf',SVC(random_state=1))])
param_grid = [{'clf__C':params_range,'clf__kernel':['linear']},
             {'clf__C':params_range,'clf__kernel':['rbf'],'clf__gamma':params_range}]
gs = GridSearchCV(estimator=pipe_svc,param_grid=param_grid,scoring='accuracy',cv=10,n_jobs=-1)
gs.fit(X_train,y_train)
print(gs.best_score_)
print(gs.best_params_)

0.826645264848
{'clf__gamma': 0.01, 'clf__C': 100, 'clf__kernel': 'rbf'}
